import libraries, assign column names for pandas, search url for specific class and capute that data

In [38]:
# import libraries
import json, requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="capstone project app")

# assign column names
columns = ['PostalCode', 'Borough', 'Neighborhood']

# get html from url using lxml parser and return only data within the wikitable sortable class
resp = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(resp, 'lxml')
data = soup.find('table',{'class':'wikitable sortable'})

iterate through captured data and place each row into array as an array

In [39]:
# iterate through rows and save as multideminsional array
sites = []
for row in data.findAll('tr')[1:]:
    cells = row.find_all(['td', 'th'])
   
    try:
        postalCode, borough, neighborhood = cells[0].text, cells[1].text, cells[2].text
    except IndexError:
        continue

    sites.append([postalCode, borough, neighborhood.rstrip()])

create dataFrame, remove Not assigned Borough and reassign Not assigned Neighborhood with Borough

In [40]:
# create dataFrame from data and assign column names
df = pd.DataFrame(sites, columns=columns)

# drop Not assigned rows for Borough
df = df[~df['Borough'].isin(['Not assigned'])]
# df = df[~df['Neighborhood'].isin(['Not assigned'])]

# assign Borough to Neighborhood that has Not assigned
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']


groupby PostalCode and Borough and apply join on Neighborhood. convert series to dataFrame and reset index.

In [41]:
# groupby PostalCode and apply join to Neighborhood. Convert series to dataFrame and reset index
df = df.groupby(['PostalCode', 'Borough'], sort=False)['Neighborhood'].apply(', '.join).to_frame().reset_index(drop=False)

Geocoder/geopy do not function correctly so skipped

In [51]:
# loop until you get the coordinates
# location = geolocator.geocode('M5G, Toronto, Ontario')
# print((location.latitude, location.longitude))

'''
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-24-45cd5e388274> in <module>()
      1 # loop until you get the coordinates
      2 location = geolocator.geocode('M5G, Toronto, Ontario')
----> 3 print((location.latitude, location.longitude))

AttributeError: 'NoneType' object has no attribute 'latitude'
'''

"\n---------------------------------------------------------------------------\nAttributeError                            Traceback (most recent call last)\n<ipython-input-24-45cd5e388274> in <module>()\n      1 # loop until you get the coordinates\n      2 location = geolocator.geocode('M5G, Toronto, Ontario')\n----> 3 print((location.latitude, location.longitude))\n\nAttributeError: 'NoneType' object has no attribute 'latitude'\n"

Grab coordinates from csv file and merge into one dataFrame (df)

In [54]:
latlong = pd.read_csv('Geospatial_Coordinates.csv')
latlong.columns = ['PostalCode','Latitude','Longitude']
df = pd.merge(df, latlong, on='PostalCode')

In [55]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
